In [ ]:
# Organizing Input Data
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
with_mask_data = os.listdir('/kaggle/input/face-mask-dataset/data/with_mask/')

In [ ]:
without_mask_data = os.listdir('/kaggle/input/face-mask-dataset/data/without_mask/')

In [ ]:
print("Number of with mask Images:", len(with_mask_data))
print("Number of without mask Images:", len(without_mask_data))

In [ ]:
# Creating labels for with and without dataset
with_mask_labels = [1]*3725
without_mask_labels = [0]*3828
total_labels = with_mask_labels + without_mask_labels

In [ ]:
# Displaying any image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
mask_img = mpimg.imread('/kaggle/input/face-mask-dataset/data/with_mask/with_mask_1.jpg')
unmask_img = mpimg.imread('/kaggle/input/face-mask-dataset/data/without_mask/without_mask_1.jpg')

In [ ]:
imgplot1 = plt.imshow(mask_img)
plt.show()

In [ ]:
imgplot2 = plt.imshow(unmask_img)
plt.show()

In [ ]:
# Processing the Images : resizing and converting to numpy arrays
from PIL import Image
import numpy as np

data = []
img_path1 = '/kaggle/input/face-mask-dataset/data/with_mask/'
for img_files in with_mask_data:
    image = Image.open(img_path1 + img_files)
    image = image.resize((128,128))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)
    
img_path2 = '/kaggle/input/face-mask-dataset/data/without_mask/'
for img_files in without_mask_data:
    image = Image.open(img_path2 + img_files)
    image = image.resize((128,128))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)

In [ ]:
len(data)

In [ ]:
# Dependent and Independent variables

X = np.array(data)
y = np.array(total_labels)

In [ ]:
# Train-Test split and rescaling the Independent variables
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [ ]:
# Building the convolutional Neural networks
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential()

model.add(keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(128,128,3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))


model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))


model.add(keras.layers.Dense(2, activation='sigmoid'))

In [ ]:
# Compiling the Neural Network
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
# Training the Neural Network
history = model.fit(X_train_scaled, y_train, validation_split=0.1, epochs=5)

In [ ]:
# Model Evaluation
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("The Accuracy of the model is : ", accuracy)

In [ ]:
#Plotting the loss and accuracy
h = history

# plot the loss value
plt.plot(h.history['loss'], label='train loss')
plt.plot(h.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

# plot the accuracy value
plt.plot(h.history['acc'], label='train accuracy')
plt.plot(h.history['val_acc'], label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
# Making predictions
import cv2
input_image_path = input('Path of the image to be predicted: ')
input_image = cv2.imread(input_image_path)
input_image_resized = cv2.resize(input_image, (128,128))
input_image_scaled = input_image_resized/255
input_image_reshaped = np.reshape(input_image_scaled, [1,128,128,3])
input_prediction = model.predict(input_image_reshaped)
print(input_prediction)
input_pred_label = np.argmax(input_prediction)
print(input_pred_label)
cv2.imshow("Input Image", input_image_path)

if input_pred_label == 1:
  print('The person in the image is wearing a mask')
else:
  print('The person in the image is not wearing a mask')